Esse notebook é uma tradução do [**artigo**](https://towardsdatascience.com/a-complete-machine-learning-project-walk-through-in-python-part-two-300f1f8147e2) originalmente escrito por [**William Koehrsen**](https://twitter.com/koehrsen_will) no site [**Towards Data Science**](https://towardsdatascience.com)

# Um projeto completo de aprendizado de máquina em Python - Parte 2